In [1]:
import time

from pylsl import StreamInlet, resolve_stream
from brainflow.data_filter import DataFilter, WindowOperations, WaveletTypes, AggOperations, WindowOperations, WaveletDenoisingTypes, WaveletExtensionTypes, NoiseEstimationLevelTypes, ThresholdTypes

import numpy as np
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pygame.mixer
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

pygame 2.5.2 (SDL 2.28.3, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.mixer.init()

# Helper Functions

In [3]:
def stream_data(duration):
    '''
    Input: duration (in seconds) of the EEG recording.

    Output: A numpy array of size or shape (channel, timesteps). Where channel is 4 for Muse2
        and timestep varies for each recording.
    '''


    # PyLSL to collect the EEG data streamed from BlueMuse.
    # Code adapted from: https://github.com/chkothe/pylsl/blob/master/examples/ReceiveData.py
    streams = resolve_stream('type', 'EEG')
    inlet = StreamInlet(streams[0])
    data = []

    # This captures data for a specified duration (in seconds).
    start_time = time.time()
    while time.time() - start_time < duration:    
        sample, timestamp = inlet.pull_sample()    
        data.append(sample)
    data_array = np.array(data).T
    data_array = np.ascontiguousarray(data_array)
    return data_array
    

In [4]:
def pad_or_trim(data, target_length):
    '''
    Input: data (coming from EEG recording) with shape (channels, timesteps).
            target_length to either pad or trim the timesteps to the desired level.
        
    Output: A modified version of data which the timesteps trimmed.

    For example, recording with Muse2 can yield 250 - 265 timesteps due to certain inconsistencies.
    We can use this function to always trim/pad the data recorded and limit it to 256 timesteps (From (4, 260) to (4, 256))

    '''
    if data.shape[1] < target_length:
        pad_width = target_length - data.shape[1]
        data = np.pad(data, ((0, 0), (0, pad_width)), 'constant', constant_values=0)
    else:
        data = data[:, :target_length]
    return data

## Muse2 Board

### Board Initialisation

This section has been removed as Brainflow had connectivity issues. Streaming is now done through BlueMuse application and PyLSL library.

### Parameters for Data Collection

In [5]:
# This is so we don't overwrite the entire data when we re-run the collection.
session_features = None
session_labels = None
session_subjects = None
save = False
session_start = datetime.now().strftime('%d-%m-%Y %H_%M')

In [8]:
done_sfx = pygame.mixer.Sound('Sound Effects/Enter.mp3')
sampling_freq = 256 # Muse2's sampling frequency. (How many timesteps per second)

duration = 2 # How long should each sample be? (in seconds)
rest_time = 1 # How many seconds in between each recording?

In [151]:
iterations = 5
midi_note = 'E4' # Initially, let's just try 0 1 2 3 for do re mi (think of the sound) (C4, D4, E4) 0 is doing something else.
subject_name = 'Taylor'

In [175]:
session_features.shape

(400, 4, 512)

### Data Collection

In [174]:
print("Be sure to only blink during the rest times allocated.")

for i in range(iterations):
    print("Iteration ", i + 1, "/", iterations)
    print("Think of the note: ", midi_note, "in ", rest_time, " second(s).")
    time.sleep(rest_time)

    ## Streaming ##
    data = stream_data(duration=duration)
    if done_sfx:
        done_sfx.play()

    ## End Stream ## 
    ## Start Data Storage ## 
    
    # This pads/trims the data so it's 256 timesteps long (or whatever the sampling frequency is)
    data = pad_or_trim(data, sampling_freq*duration) # -> (channels, sampling_freq) shape. Or (4, 256) for Muse2. [EEG only]

    data = np.expand_dims(data, 0)[:, 0:4]
    label = np.array([midi_note]) # Saves the current note as a feature.
    subject = np.array([subject_name])
    if session_features is None:
        session_features = data 
        session_labels = label
        session_subjects = subject
    else:
        session_features = np.append(session_features, data, axis=0)
        session_labels = np.append(session_labels, label, axis=0)
        session_subjects = np.append(session_subjects, subject, axis=0)

    
    # This clears the Python notebook output.
    clear_output()

### Data Processing

In [176]:
# Let's only use the 4 EEG channels.
session_features = session_features[:, 0:4, :]

In [177]:
session_features_denoised = session_features.copy()
session_features_decomposed = None

In [178]:
# We would have to denoise the data.
### TO IMPLEMENT: Data denoising.
# Refer to this: https://brainflow.readthedocs.io/en/stable/Examples.html#python-denoising
# Note: This function performs denoising IN PLACE.
nfft = DataFilter.get_nearest_power_of_two(sampling_freq) # This is used to extract the band power.

for idx in range(len(session_features_denoised)):
    for channel in range(len(session_features_denoised[idx])):
        # This part performs the denoising INPLACE.
        DataFilter.perform_wavelet_denoising(session_features_denoised[idx][channel],
                                     WaveletTypes.BIOR3_9,
                                     3,
                                     WaveletDenoisingTypes.SURESHRINK,
                                     ThresholdTypes.HARD,
                                     WaveletExtensionTypes.SYMMETRIC,
                                     NoiseEstimationLevelTypes.FIRST_LEVEL
                                     )

In [179]:
for idx in range(len(session_features_denoised)):
    observation_band_powers = None
    for channel in range(len(session_features_denoised[idx])):
        # Next, we'd also have to extract the different wave states from each channel.
        # Refer to this: https://brainflow.readthedocs.io/en/stable/Examples.html#python-band-power

        psd = DataFilter.get_psd_welch(session_features_denoised[idx][channel], nfft, nfft // 2, sampling_freq,
                                WindowOperations.BLACKMAN_HARRIS)

        band_power_delta =  DataFilter.get_band_power(psd, 0.5, 4.0)
        band_power_theta =  DataFilter.get_band_power(psd, 4.1, 8.0)
        band_power_alpha = DataFilter.get_band_power(psd, 8.1, 12.0)
        band_power_beta = DataFilter.get_band_power(psd, 12.1, 30.0)
        band_power_gamma = DataFilter.get_band_power(psd, 14.1, 30.0)

        band_power = np.array([band_power_delta, band_power_theta, band_power_alpha, band_power_beta, band_power_gamma])
        # band_power is a numpy array of shape [1, 5]
        # For each observation, we want an array of shape [5, 5]
        # The first five is for the channels: TP9, AF7, AF8, TP10, RIGHT AUX
        if observation_band_powers is None:
            observation_band_powers = band_power
        else:
            observation_band_powers = np.append(observation_band_powers, band_power, axis=0)
    observation_band_powers = np.expand_dims(observation_band_powers, axis=0)
    if session_features_decomposed is None:
        session_features_decomposed = observation_band_powers
    else:
        session_features_decomposed = np.append(session_features_decomposed, observation_band_powers, axis=0)

### Data Saving

In [180]:
# Save the data as a .npy file
if save:
    np.save(f'{session_start} RAW.npy', session_features)
    np.save(f'{session_start} DECOMPOSED.npy', session_features_decomposed)
    np.save(f'{session_start} DENOISED.npy', session_features_denoised)
    np.save(f'{session_start} LABELS.npy', session_labels)
    np.save(f'{session_start} SUBJECTS.npy', session_subjects)

## Converting to DataFrame Format

First, let's convert our decomposed data into a pandas dataframe. We start by generating the column names for the data frame. Note that the order in which the column names are created is crucial because we want to label the respective PSD of each brainwave type from each channel.

In [181]:
channel_names = ['TP9', 'AF7', 'AF8', 'TP10']
brain_waves_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']
colnames = []
for channel in channel_names:
    for wave in brain_waves_names:
        colnames.append(f"{channel}_{wave}")



In [182]:
df = pd.DataFrame(session_features_decomposed, columns=colnames)
df['target'] = session_labels
df['subject'] = session_subjects

In [183]:
features = df.columns.difference(['target', 'subject'])
X = df[features]
y = df['target']

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [185]:
rfClassifier = RandomForestClassifier()
rfClassifier.fit(X_train, y_train)

RandomForestClassifier()

In [186]:
# Make predictions on the test set
y_pred = rfClassifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')


Accuracy: 0.7375
Confusion Matrix:
[[15  0  0  5]
 [ 1 11  8  0]
 [ 1  5 14  0]
 [ 0  0  1 19]]
Classification Report:
              precision    recall  f1-score   support

          C4       0.88      0.75      0.81        20
          D4       0.69      0.55      0.61        20
          E4       0.61      0.70      0.65        20
     Nothing       0.79      0.95      0.86        20

    accuracy                           0.74        80
   macro avg       0.74      0.74      0.73        80
weighted avg       0.74      0.74      0.73        80



In [188]:
feature_importance_df = pd.DataFrame({
    'feature': features,
    'importance': rfClassifier.feature_importances_
}).sort_values(by='importance', ascending=False)

In [189]:
feature_importance_df

,feature,importance
6,AF8_beta,0.119024
7,AF8_delta,0.100051
8,AF8_gamma,0.098816
5,AF8_alpha,0.085842
13,TP10_gamma,0.070798
11,TP10_beta,0.060215
9,AF8_theta,0.043913
3,AF7_gamma,0.043536
1,AF7_beta,0.039780
12,TP10_delta,0.039742
